# DeltaLake Data Analysis with DuckDB

This notebook demonstrates how to analyze the timeseries data stored in DeltaLake format using DuckDB and create visualizations.

## Setup and Imports

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## Connect to DeltaLake Data

In [ ]:
# Connect to DuckDB and install delta extension
conn = duckdb.connect()
conn.execute("INSTALL delta")
conn.execute("LOAD delta")

print("Connected to DuckDB with Delta extension loaded!")

## Query DeltaLake Tables

In [ ]:
# Define path to delta tables
delta_dir = "../../../output/timeseries-delta"
delta_path = Path(delta_dir).absolute()

print(f"Delta directory: {delta_path}")
print(f"Directory exists: {delta_path.exists()}")

In [ ]:
# Query all parcel data from DeltaLake tables
# Note: We need to query each chunk individually since wildcard patterns don't work with delta_scan

# Find all parcel chunk directories
chunk_dirs = list(delta_path.glob("parcel_chunk=*"))
print(f"Found {len(chunk_dirs)} parcel chunks to query")

# Query each chunk and combine
dataframes = []
for chunk_dir in sorted(chunk_dirs):
    try:
        query = f"""
        SELECT 
            parcel_id,
            date,
            utm_tile,
            ndvi,
            evi,
            savi,
            cloud_cover,
            observation_quality
        FROM delta_scan('{chunk_dir}')
        """
        chunk_df = conn.execute(query).df()
        if not chunk_df.empty:
            dataframes.append(chunk_df)
            print(f"Loaded {len(chunk_df):,} records from {chunk_dir.name}")
    except Exception as e:
        print(f"Warning: Failed to read {chunk_dir.name}: {e}")
        continue

# Combine all dataframes
df = pd.concat(dataframes, ignore_index=True)
df = df.sort_values(['parcel_id', 'date']).reset_index(drop=True)

print(f"Total loaded: {len(df):,} observations for {df['parcel_id'].nunique():,} parcels")
df.head()

## Data Overview and Statistics

In [ ]:
# Basic data information
print("=== DATASET OVERVIEW ===")
print(f"Total observations: {len(df):,}")
print(f"Unique parcels: {df['parcel_id'].nunique():,}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"UTM tiles: {df['utm_tile'].unique()}")
print(f"Observation dates: {df['date'].nunique()}")

# Data types and missing values
print("\n=== DATA INFO ===")
print(df.info())

In [ ]:
# Statistical summary
print("=== STATISTICAL SUMMARY ===")
df.describe()

## Vegetation Index Distributions

In [ ]:
# Create distribution plots for vegetation indices
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# NDVI distribution
axes[0, 0].hist(df["ndvi"], bins=50, alpha=0.7, color='green', edgecolor='black')
axes[0, 0].set_title("NDVI Distribution", fontsize=14)
axes[0, 0].set_xlabel("NDVI")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].axvline(df["ndvi"].mean(), color='red', linestyle='--', label=f'Mean: {df["ndvi"].mean():.3f}')
axes[0, 0].legend()

# EVI distribution
axes[0, 1].hist(df["evi"], bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0, 1].set_title("EVI Distribution", fontsize=14)
axes[0, 1].set_xlabel("EVI")
axes[0, 1].set_ylabel("Frequency")
axes[0, 1].axvline(df["evi"].mean(), color='red', linestyle='--', label=f'Mean: {df["evi"].mean():.3f}')
axes[0, 1].legend()

# SAVI distribution
axes[1, 0].hist(df["savi"], bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1, 0].set_title("SAVI Distribution", fontsize=14)
axes[1, 0].set_xlabel("SAVI")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].axvline(df["savi"].mean(), color='red', linestyle='--', label=f'Mean: {df["savi"].mean():.3f}')
axes[1, 0].legend()

# Cloud cover distribution
axes[1, 1].hist(df["cloud_cover"], bins=30, alpha=0.7, color='gray', edgecolor='black')
axes[1, 1].set_title("Cloud Cover Distribution", fontsize=14)
axes[1, 1].set_xlabel("Cloud Cover (%)")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].axvline(df["cloud_cover"].mean(), color='red', linestyle='--', label=f'Mean: {df["cloud_cover"].mean():.1f}%')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## Correlation Analysis

In [ ]:
# Correlation matrix
numeric_cols = ["ndvi", "evi", "savi", "cloud_cover", "observation_quality"]
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.3f', cbar_kws={"shrink": .8})
plt.title("Correlation Matrix: Vegetation Indices and Environmental Factors", fontsize=16)
plt.tight_layout()
plt.show()

print("Correlation Matrix:")
print(corr_matrix.round(3))

## Time Series Analysis

In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Sample a few parcels for time series visualization
sample_parcels = df['parcel_id'].unique()[:15]
sample_data = df[df['parcel_id'].isin(sample_parcels)]

plt.figure(figsize=(15, 8))

for i, parcel_id in enumerate(sample_parcels):
    parcel_data = sample_data[sample_data['parcel_id'] == parcel_id].sort_values('date')
    plt.plot(parcel_data['date'], parcel_data['ndvi'], 
             marker='o', markersize=4, alpha=0.8, linewidth=2)

plt.title(f"NDVI Time Series for {len(sample_parcels)} Sample Parcels", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("NDVI", fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Monthly Trends Analysis

In [ ]:
# Add month information
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime('%B')

# Calculate monthly averages
monthly_stats = df.groupby(['month', 'month_name']).agg({
    'ndvi': ['mean', 'std'],
    'evi': ['mean', 'std'],
    'savi': ['mean', 'std'],
    'cloud_cover': ['mean', 'std']
}).round(4)

print("Monthly Statistics:")
print(monthly_stats)

In [ ]:
# Plot monthly trends
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

monthly_means = df.groupby('month_name')[['ndvi', 'evi', 'savi', 'cloud_cover']].mean()
months_order = df.sort_values('month')['month_name'].unique()
monthly_means = monthly_means.reindex(months_order)

# NDVI trend
axes[0, 0].plot(months_order, monthly_means['ndvi'], marker='o', linewidth=3, markersize=8, color='green')
axes[0, 0].set_title("Monthly Average NDVI", fontsize=14)
axes[0, 0].set_ylabel("NDVI")
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# EVI trend
axes[0, 1].plot(months_order, monthly_means['evi'], marker='o', linewidth=3, markersize=8, color='blue')
axes[0, 1].set_title("Monthly Average EVI", fontsize=14)
axes[0, 1].set_ylabel("EVI")
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# SAVI trend
axes[1, 0].plot(months_order, monthly_means['savi'], marker='o', linewidth=3, markersize=8, color='orange')
axes[1, 0].set_title("Monthly Average SAVI", fontsize=14)
axes[1, 0].set_ylabel("SAVI")
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# Cloud cover trend
axes[1, 1].plot(months_order, monthly_means['cloud_cover'], marker='o', linewidth=3, markersize=8, color='gray')
axes[1, 1].set_title("Monthly Average Cloud Cover", fontsize=14)
axes[1, 1].set_ylabel("Cloud Cover (%)")
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## UTM Tile Comparison

In [ ]:
# Compare vegetation indices between UTM tiles
tile_comparison = df.groupby('utm_tile').agg({
    'ndvi': ['mean', 'std', 'count'],
    'evi': ['mean', 'std'],
    'savi': ['mean', 'std'],
    'cloud_cover': ['mean', 'std'],
    'parcel_id': 'nunique'
}).round(4)

print("UTM Tile Comparison:")
print(tile_comparison)

In [ ]:
# Box plots comparing tiles
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# NDVI by tile
df.boxplot(column='ndvi', by='utm_tile', ax=axes[0, 0])
axes[0, 0].set_title("NDVI Distribution by UTM Tile")
axes[0, 0].set_xlabel("UTM Tile")
axes[0, 0].set_ylabel("NDVI")

# EVI by tile
df.boxplot(column='evi', by='utm_tile', ax=axes[0, 1])
axes[0, 1].set_title("EVI Distribution by UTM Tile")
axes[0, 1].set_xlabel("UTM Tile")
axes[0, 1].set_ylabel("EVI")

# Cloud cover by tile
df.boxplot(column='cloud_cover', by='utm_tile', ax=axes[1, 0])
axes[1, 0].set_title("Cloud Cover Distribution by UTM Tile")
axes[1, 0].set_xlabel("UTM Tile")
axes[1, 0].set_ylabel("Cloud Cover (%)")

# Observation quality by tile
df.boxplot(column='observation_quality', by='utm_tile', ax=axes[1, 1])
axes[1, 1].set_title("Observation Quality by UTM Tile")
axes[1, 1].set_xlabel("UTM Tile")
axes[1, 1].set_ylabel("Observation Quality")

plt.suptitle("")  # Remove default title
plt.tight_layout()
plt.show()

## Advanced Queries with DuckDB

In [ ]:
# Example: Find parcels with highest NDVI growth
# Note: Using the already loaded dataframe instead of direct DuckDB query for simplicity

# Calculate NDVI growth per parcel
parcel_stats = df.groupby(['parcel_id', 'utm_tile']).agg({
    'ndvi': ['min', 'max', 'mean', 'count']
}).round(4)

# Flatten column names
parcel_stats.columns = ['min_ndvi', 'max_ndvi', 'avg_ndvi', 'observations']
parcel_stats = parcel_stats.reset_index()

# Calculate NDVI growth
parcel_stats['ndvi_growth'] = parcel_stats['max_ndvi'] - parcel_stats['min_ndvi']

# Filter parcels with at least 10 observations and sort by growth
high_growth = parcel_stats[parcel_stats['observations'] >= 10].sort_values('ndvi_growth', ascending=False).head(10)

print("Top 10 Parcels with Highest NDVI Growth:")
print(high_growth)

In [ ]:
# Example: Monthly aggregations using pandas
# Since we already have the data loaded, we can use pandas for aggregations

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

monthly_agg_df = df.groupby(['year', 'month', 'utm_tile']).agg({
    'parcel_id': ['count', 'nunique'],
    'ndvi': ['mean', 'min', 'max'],
    'evi': 'mean',
    'cloud_cover': 'mean'
}).round(4)

# Flatten column names
monthly_agg_df.columns = [
    'observations', 'unique_parcels', 'avg_ndvi', 'min_ndvi', 'max_ndvi', 
    'avg_evi', 'avg_cloud_cover'
]
monthly_agg_df = monthly_agg_df.reset_index()

print("Monthly Aggregations by UTM Tile:")
print(monthly_agg_df)

## Data Quality Assessment

In [ ]:
# Check for data quality issues
print("=== DATA QUALITY ASSESSMENT ===")

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Check for outliers (values outside reasonable ranges)
print("\nPotential outliers:")
print(f"NDVI < -1 or > 1: {((df['ndvi'] < -1) | (df['ndvi'] > 1)).sum()}")
print(f"EVI < -1 or > 1: {((df['evi'] < -1) | (df['evi'] > 1)).sum()}")
print(f"Cloud cover < 0 or > 100: {((df['cloud_cover'] < 0) | (df['cloud_cover'] > 100)).sum()}")

# Check observation counts per parcel
obs_counts = df.groupby('parcel_id').size()
print(f"\nObservations per parcel:")
print(f"Min: {obs_counts.min()}")
print(f"Max: {obs_counts.max()}")
print(f"Mean: {obs_counts.mean():.1f}")
print(f"Parcels with < 10 observations: {(obs_counts < 10).sum()}")

## Summary and Conclusions

In [ ]:
print("=== ANALYSIS SUMMARY ===")
print(f"📊 Dataset: {len(df):,} observations across {df['parcel_id'].nunique():,} parcels")
print(f"📅 Time period: {df['date'].min()} to {df['date'].max()}")
print(f"🗺️  UTM tiles: {', '.join(df['utm_tile'].unique())}")
print(f"🌱 Average NDVI: {df['ndvi'].mean():.3f} (± {df['ndvi'].std():.3f})")
print(f"☁️  Average cloud cover: {df['cloud_cover'].mean():.1f}%")
print(f"📈 NDVI range: {df['ndvi'].min():.3f} to {df['ndvi'].max():.3f}")

# Key findings
print("\n=== KEY FINDINGS ===")
print("• Vegetation indices show seasonal patterns with growth from winter to spring")
print("• Strong positive correlation between NDVI, EVI, and SAVI (as expected)")
print("• Cloud cover varies significantly and may impact data quality")
print("• UTM tiles show some differences in vegetation characteristics")
print("• Data quality is generally good with consistent observations per parcel")

# Clean up
conn.close()
print("\n✅ Analysis complete!")